# Projet 08 : Communiquer des résultats
# Partie 1 : Préparation des données - Suite

---
**Importation des librairies**

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
#import warnings
#warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import seaborn as sns
import scipy.stats as st
import statsmodels.api as sm
import statsmodels.formula.api as smf

import modules_perso.sf_graphiques as sfg

<a id='1_0'></a>

In [4]:
prices = pd.read_csv("data_contract_prices.csv", sep=";", decimal=",", parse_dates=["dateref"], index_col=[0,1])
prices.head()

close_price  mid_price  real_price
contract_id dateref                                       
1           1989-11-01        28.12     28.070       75.50
            1989-11-02        27.79     27.905       75.17
            1989-11-03        28.05     27.935       75.43
            1989-11-06        28.45     28.360       75.83
            1989-11-07        28.52     28.570       75.90

In [5]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 360539 entries, (1, 1989-11-01 00:00:00) to (92, 2019-11-14 00:00:00)
Data columns (total 3 columns):
close_price    360539 non-null float64
mid_price      360539 non-null float64
real_price     360539 non-null float64
dtypes: float64(3)
memory usage: 9.3 MB


### Préparation d'un dataframe avec l'évolution future des prix à partir d'une date donnée

Nos signaux sont calculés sur la base des données disponibles à la fin de la journée de cotation d'un produit. Pour évaluer leur intérêt, il ne faut pas prendre en compte cette journée.  
On peut évaluer l'évolution des prix en calculant la variation en pourcentage entre le prix de clôture N journées plus tard par rapport au prix de clôture du jour. La variation sur un mois peut être estimée en prenant N = 21 jours de marché, celle sur deux mois par N = 42 jours de marché, et celle sur trois mois par N = 64 jours de marché.  
  
Il faut ajuster la variation du prix par la volatilité du marché. En effet, un gain de 5% est plus significatif sur un marché dont la volatilité est de 5% que sur un marché dont la volatilité est de 15%. La volatilité est mesurée par l'écart-type des variations journalières en pourcentage du prix de marché. Elle est en général annualisée : si on utilise des données journalières, ce qui est notre cas, la volatilité obtenue est celle journalière. Pour obtenir la volatilité annualisée, il faut la multiplier par la racine carrée du nombre de jours de marché dans une année. Nous multiplierons par la racine carrée de 256 jours de marché (4 * 64 jours de marché par trimestre comme stipulé avant), soit par 16.  
  
Petite note sur les prix disponibles dans notre dataframe :  
- le close price est le prix de clôture du marché corrigé des écarts qui existent entre deux échéances lorsque l'échéance prise en compte change ;  
- le mid price est le cours moyen du marché sur la même base que le close price ;  
- le real price est le prix de clôture du marché non corrigé des écarts entre échéance.  
  
La variation de prix en valeur sur une période donnée est donc la différence entre le close price de la dernière journée et le close price de la première journée.  
La variation de prix en pourcentage est égale à la variation de prix en valeur sur la période divisée par le real price de la première journée.  
  
A noter que les données sont déjà triées par contract_id et par date lors de la création du fichier csv et donc dans le dataframe prices.

Comme je cherche à connaître l'évolution future du prix, j'utilise des périodes négatives.

In [6]:
prices_evolution = prices.reset_index()
prices_evolution["variation_01_pourcent"] = -1.0 * prices_evolution.groupby("contract_id").close_price.diff(periods=-1) / \
    prices_evolution.groupby("contract_id").real_price.shift(0)
prices_evolution["variation_21_pourcent"] = -1.0 * prices_evolution.groupby("contract_id").close_price.diff(periods=-21) / \
    prices_evolution.groupby("contract_id").real_price.shift(0)
prices_evolution["variation_42_pourcent"] = -1.0 * prices_evolution.groupby("contract_id").close_price.diff(periods=-42) / \
    prices_evolution.groupby("contract_id").real_price.shift(0)
prices_evolution["variation_64_pourcent"] = -1.0 * prices_evolution.groupby("contract_id").close_price.diff(periods=-64) / \
    prices_evolution.groupby("contract_id").real_price.shift(0)

In [7]:
prices_evolution["volatilite_21"] = 16 * prices_evolution.variation_01_pourcent.rolling(21).std(ddof=1).shift(-20)
prices_evolution["volatilite_42"] = 16 * prices_evolution.variation_01_pourcent.rolling(42).std(ddof=1).shift(-41)
prices_evolution["volatilite_64"] = 16 * prices_evolution.variation_01_pourcent.rolling(64).std(ddof=1).shift(-63)
prices_evolution["variation_21_standard"] = prices_evolution.variation_21_pourcent / prices_evolution.volatilite_21
prices_evolution["variation_42_standard"] = prices_evolution.variation_42_pourcent / prices_evolution.volatilite_42
prices_evolution["variation_64_standard"] = prices_evolution.variation_64_pourcent / prices_evolution.volatilite_64
prices_evolution.head(50)

,contract_id,dateref,close_price,mid_price,real_price,variation_01_pourcent,variation_21_pourcent,variation_42_pourcent,variation_64_pourcent,volatilite_21,volatilite_42,volatilite_64,variation_21_standard,variation_42_standard,variation_64_standard
0,1,1989-11-01,28.12,28.070,75.50,-0.004371,0.005298,0.026358,0.005166,0.064441,0.069941,0.090376,0.082215,0.376857,0.057156
1,1,1989-11-02,27.79,27.905,75.17,0.003459,0.015565,0.024079,0.009445,0.064867,0.071130,0.089921,0.239948,0.338517,0.105039
2,1,1989-11-03,28.05,27.935,75.43,0.005303,0.012727,0.023465,-0.004242,0.064099,0.070993,0.091989,0.198553,0.330530,-0.046118
3,1,1989-11-06,28.45,28.360,75.83,0.000923,0.006066,0.019254,-0.013715,0.062027,0.069996,0.091715,0.097800,0.275066,-0.149538
4,1,1989-11-07,28.52,28.570,75.90,0.002635,0.005665,0.022134,-0.021080,0.061990,0.070459,0.092558,0.091391,0.314145,-0.227753
5,1,1989-11-08,28.72,28.695,76.10,0.003285,0.005256,0.024704,-0.017871,0.061808,0.071200,0.093215,0.085042,0.346970,-0.191721
6,1,1989-11-09,28.97,29.070,76.35,0.002358,0.006549,0.022266,-0.026064,0.062773,0.070879,0.093422,0.104324,0.314138,-0.278993
7,1,1989-11-13,29.15,29.185,76.53,-0.002613,0.003136,0.018163,-0.034104,0.062481,0.070929,0.093896,0.050192,0.256070,-0.363213
8,1,1989-11-14,28.95,28.910,76.33,-0.005633,0.006157,0.018865,-0.021879,0.061651,0.070764,0.096123,0.099876,0.266596,-0.227611
9,1,1989-11-15,28.52,28.570,75.90,-0.001845,0.007246,0.028458,-0.009223,0.060428,0.069499,0.096680,0.119917,0.409483,-0.095393


In [8]:
prices_evolution.to_csv("prices_evolution.csv")